In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
print('ass')
import functions.parse_data as parse
import functions.handy_functions as hf

from multivariate_quantile_regression.anton_trial_model import QuantileNetwork
import torch.nn as nn

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

2024-02-15 08:14:24.355576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
channel_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06','Cloud_B07',
                 'Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13']

data_water=parse.parse('SMHIdata/cloudrm_water.dat')
data_clear=parse.parse('SMHIdata/cloudrm_clear.dat')
data_ice=parse.parse('SMHIdata/cloudrm_ice.dat')
data_mixed=parse.parse('SMHIdata/cloudrm_mixed.dat')

#Concatinate all datasets
data_all=pd.concat([data_water, data_clear, data_ice, data_mixed])

In [3]:
channel_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06','Cloud_B07',
                 'Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13']

data_water=parse.parse('SMHIdata/cloudrm_water.dat')
data_clear=parse.parse('SMHIdata/cloudrm_clear.dat')
data_ice=parse.parse('SMHIdata/cloudrm_ice.dat')
data_mixed=parse.parse('SMHIdata/cloudrm_mixed.dat')

#Concatinate all datasets
data_all=pd.concat([data_water, data_clear, data_ice, data_mixed])
data_all=data_all.drop(columns=['Surface_Desc','Cloud_B01','Clear_B01'])
data_all=hf.add_MSI_noise(data_all,channel_labels)

Noise standard deviation for Cloud_B02: 0.00335001428051948
Noise standard deviation for Cloud_B03: 0.002912530185416667
Noise standard deviation for Cloud_B04: 0.004058081082042254
Noise standard deviation for Cloud_B05: 0.0046524891611111115
Noise standard deviation for Cloud_B06: 0.007455351321348316
Noise standard deviation for Cloud_B07: 0.008871707484285717
Noise standard deviation for Cloud_B08: 0.04489677938000001
Noise standard deviation for Cloud_B09: 0.005688141120114942
Noise standard deviation for Cloud_B10: 0.003909328971491229
Noise standard deviation for Cloud_B11: 0.0014014724139999996
Noise standard deviation for Cloud_B12: 0.005030040539999999
Noise standard deviation for Cloud_B13: 0.004041267081999999


In [4]:
##Train test validation split##
X_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
           'Cloud_B07','Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13',
           'Sat_Zenith_Angle','Sun_Zenith_Angle','Azimuth_Diff_Angle']

#Leave out 'GOT', 'Water_Vapor'
#Band 1 no go.
y_labels=['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
           'Clear_B07','Clear_B08','Clear_B09','Clear_B10','Clear_B11','Clear_B12','Clear_B13']

df=hf.normalise_input_df(data_all,X_labels)
df=hf.add_noise(df,X_labels,sigma=0.001)

##Split data##
X=df[X_labels]
y=df[y_labels]

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.05)

In [5]:
import random

val_size=0.05

validation_indices1=np.array(random.sample(range(len(X_train['Cloud_B02'])), int(len(X_train['Cloud_B02'])*val_size)))
train_indices1=[i for i in range(len(X_train['Cloud_B02'])) if np.any(validation_indices1==i)==False]

validation_indices2=np.array(random.sample(range(len(X_train['Cloud_B02'])), int(len(X_train['Cloud_B02'])*val_size)))
train_indices2=[i for i in range(len(X_train['Cloud_B02'])) if np.any(validation_indices2==i)==False]

validation_indices3=np.array(random.sample(range(len(X_train['Cloud_B02'])), int(len(X_train['Cloud_B02'])*val_size)))
train_indices3=[i for i in range(len(X_train['Cloud_B02'])) if np.any(validation_indices3==i)==False]

validation_indices4=np.array(random.sample(range(len(X_train['Cloud_B02'])), int(len(X_train['Cloud_B02'])*val_size)))
train_indices4=[i for i in range(len(X_train['Cloud_B02'])) if np.any(validation_indices4==i)==False]

validation_indices5=np.array(random.sample(range(len(X_train['Cloud_B02'])), int(len(X_train['Cloud_B02'])*val_size)))
train_indices5=[i for i in range(len(X_train['Cloud_B02'])) if np.any(validation_indices5==i)==False]

In [6]:
quantiles=np.array([0.1,0.5,0.9])
batch_size=100
nepochs=20

seq1 = lambda: nn.Sequential(
            nn.Linear(np.shape(X_train)[1],128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128, np.shape(y_train)[1]*np.size(quantiles))
            )

seq2 = lambda: nn.Sequential(
            nn.Linear(np.shape(X_train)[1],128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128, np.shape(y_train)[1]*np.size(quantiles))
            )

model11=QuantileNetwork(quantiles=quantiles)
model11.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices1,validation_indices1,batch_size=batch_size,nepochs=nepochs,sequence=seq1())

model12=QuantileNetwork(quantiles=quantiles)
model12.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices2,validation_indices2,batch_size=batch_size,nepochs=nepochs,sequence=seq1())

model13=QuantileNetwork(quantiles=quantiles)
model13.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices3,validation_indices3,batch_size=batch_size,nepochs=nepochs,sequence=seq1())

model14=QuantileNetwork(quantiles=quantiles)
model14.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices4,validation_indices4,batch_size=batch_size,nepochs=nepochs,sequence=seq1())

model15=QuantileNetwork(quantiles=quantiles)
model15.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices5,validation_indices5,batch_size=batch_size,nepochs=nepochs,sequence=seq1())


model21=QuantileNetwork(quantiles=quantiles)
model21.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices1,validation_indices1,batch_size=batch_size,nepochs=nepochs,sequence=seq2())

model22=QuantileNetwork(quantiles=quantiles)
model22.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices2,validation_indices2,batch_size=batch_size,nepochs=nepochs,sequence=seq2())

model23=QuantileNetwork(quantiles=quantiles)
model23.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices3,validation_indices3,batch_size=batch_size,nepochs=nepochs,sequence=seq2())

model24=QuantileNetwork(quantiles=quantiles)
model24.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices4,validation_indices4,batch_size=batch_size,nepochs=nepochs,sequence=seq2())

model25=QuantileNetwork(quantiles=quantiles)
model25.fit(X_train.to_numpy(),y_train.to_numpy(),train_indices5,validation_indices5,batch_size=batch_size,nepochs=nepochs,sequence=seq2())



Epoch 1


Batch number: 100%|██████████| 1805/1805 [00:06<00:00, 274.74it/s]


Training loss [0.73318523] Validation loss [0.57546157]
----New best validation loss---- [0.57546157]
Epoch 2


Batch number: 100%|██████████| 1805/1805 [01:12<00:00, 24.99it/s] 


Training loss [0.5397119] Validation loss [0.50913996]
----New best validation loss---- [0.50913996]
Epoch 3


...

Training loss [0.3993308] Validation loss [0.39889145]
----New best validation loss---- [0.39889145]
Epoch 19



Batch number: 100%|██████████| 1805/1805 [00:10<00:00, 173.64it/s]


Training loss [0.39798334] Validation loss [0.40944967]
Epoch 20


Batch number: 100%|██████████| 1805/1805 [00:14<00:00, 125.03it/s]


Training loss [0.39614403] Validation loss [0.38949808]
----New best validation loss---- [0.38949808]
Best model out of total max epochs found at epoch 20


In [7]:
preds11 = model11.predict(X_test.to_numpy())
preds12 = model12.predict(X_test.to_numpy())
preds13 = model13.predict(X_test.to_numpy())
preds14 = model14.predict(X_test.to_numpy())
preds15 = model15.predict(X_test.to_numpy())
preds1 = (preds11+preds12+preds13+preds14+preds15)/5

preds21 = model21.predict(X_test.to_numpy())
preds22 = model22.predict(X_test.to_numpy())
preds23 = model23.predict(X_test.to_numpy())
preds24 = model24.predict(X_test.to_numpy())
preds25 = model25.predict(X_test.to_numpy())
preds2 = (preds21+preds22+preds23+preds24+preds25)/5

In [8]:
## MSE ##
MSE1 = mean_squared_error(y_test.to_numpy(),preds1[:,:,1])
MSE2 = mean_squared_error(y_test.to_numpy(),preds2[:,:,1])

print(MSE1)
print(MSE2)

0.004137716774755413
0.004076608917443449


In [9]:
print('MSE for net1:')
print(mean_squared_error(y_test.to_numpy(),preds11[:,:,1]))
print(mean_squared_error(y_test.to_numpy(),preds12[:,:,1]))
print(mean_squared_error(y_test.to_numpy(),preds13[:,:,1]))
print(mean_squared_error(y_test.to_numpy(),preds14[:,:,1]))
print(mean_squared_error(y_test.to_numpy(),preds15[:,:,1]))

print('MSE for net2:')
print(mean_squared_error(y_test.to_numpy(),preds21[:,:,1]))
print(mean_squared_error(y_test.to_numpy(),preds22[:,:,1]))
print(mean_squared_error(y_test.to_numpy(),preds23[:,:,1]))
print(mean_squared_error(y_test.to_numpy(),preds24[:,:,1]))
print(mean_squared_error(y_test.to_numpy(),preds25[:,:,1]))

MSE for net1:
0.00435680310951639
0.004519743661495947
0.0044708277849642855
0.004436473532797571
0.0044717826542661765
MSE for net2:
0.004390497110713533
0.004451778541385111
0.004378418704467907
0.0044244622391165596
0.00434295507347204
